In [11]:
# load the val annotations file

import os

def get_annotations_map():
    valAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'
    valAnnotationsFile = open(valAnnotationsPath, 'r')
    valAnnotationsContents = valAnnotationsFile.read()
    valAnnotations = {}

    for line in valAnnotationsContents.splitlines():
        pieces = line.strip().split()
        valAnnotations[pieces[0]] = pieces[1]
    
    return valAnnotations


#Sys
import numpy as np
from PIL import Image


def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,64,64],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    #print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,64,64],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


   # print('finished loading test images')+str(i)

    return X_train,y_train,X_test,y_test

if __name__ == "__main__":
    import matplotlib.pyplot as plt
    
    #Keras
    import keras
    from keras.datasets import mnist
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.optimizers import RMSprop
    #from keras.regularizers import WeightRegularizer, ActivityRegularizer 
    from keras.layers.core import Dense, Dropout, Activation, Flatten
    from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
    from keras.layers.normalization import BatchNormalization 
    from keras.utils import np_utils
    from keras.optimizers import SGD
    from keras.preprocessing.image import ImageDataGenerator
    #from plotter import Plotter
    # from keras.utils.visualize_util import plot
    import h5py


    #Params
    num_classes = 10
    batch_size = 32
    nb_epoch = 10

    path='./tiny-imagenet-200'
    X_train,y_train,X_test,y_test=load_images(path,num_classes)
    
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')
    
    num_samples=len(X_train)

    # input image dimensions
    num_channels , img_rows, img_cols = X_train.shape[1], X_train.shape[2], X_train.shape[3]

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, num_classes)
    Y_test = np_utils.to_categorical(y_test, num_classes)
    
    #train
    model = Sequential()
    #conv-spatial batch norm - relu #1 
    model.add(ZeroPadding2D((2,2),input_shape=(3,64,64)))
    model.add(Convolution2D(32,5,5,subsample=(2,2), activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32,3,3,subsample=(1,1), activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, axis=1, momentum=0.9))
    
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))


    print()
    print()
    print('===========================')
    print('Testing: with loss function hinge')
    print('===========================')
    print()


    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.summary()
    
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
              verbose=1, validation_data=(X_test, Y_test))



    score = model.evaluate(X_test, Y_test, verbose=1)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Loading 10 classes
loading training images...
loading test images...
X_train shape: (5000, 3, 64, 64)
5000 train samples
500 test samples


C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:134: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), activation="relu", strides=(2, 2))`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:135: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:139: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", strides=(1, 1))`
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:140: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)`




Testing: with loss function hinge

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_6 (ZeroPaddin (None, 7, 68, 64)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 2, 32, 32)         51232     
_________________________________________________________________
batch_normalization_5 (Batch (None, 2, 32, 32)         8         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 16, 32)         0         
_________________________________________________________________
zero_padding2d_7 (ZeroPaddin (None, 3, 18, 32)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 16, 32)         9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 1, 

C:\Program Files\Anaconda3\lib\site-packages\keras\models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 5000 samples, validate on 500 samples
Epoch 1/10
5000/5000 [==============================] - 5s 901us/step - loss: 2.3499 - acc: 0.1174 - val_loss: 2.2842 - val_acc: 0.1220
Epoch 2/10
5000/5000 [==============================] - 3s 661us/step - loss: 2.2806 - acc: 0.1312 - val_loss: 2.2585 - val_acc: 0.1540
Epoch 3/10
5000/5000 [==============================] - 3s 663us/step - loss: 2.2131 - acc: 0.1718 - val_loss: 2.1272 - val_acc: 0.2000
Epoch 4/10
5000/5000 [==============================] - 3s 664us/step - loss: 2.1465 - acc: 0.2024 - val_loss: 2.1595 - val_acc: 0.1960
Epoch 5/10
5000/5000 [==============================] - 3s 679us/step - loss: 2.1154 - acc: 0.2170 - val_loss: 2.0819 - val_acc: 0.2320
Epoch 6/10
5000/5000 [==============================] - 4s 712us/step - loss: 2.0634 - acc: 0.2426 - val_loss: 2.0401 - val_acc: 0.2560
Epoch 7/10
5000/5000 [==============================] - 4s 701us/step - loss: 2.0296 - acc: 0.2552 - val_loss: 2.0189 - val_acc: 0.2460
E